In [1]:
# Import libraries
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# Import dataset
train_df=pd.read_csv('../data/sales_train_validation.csv')

C:\Users\Yuga\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Yuga\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Yuga\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Yuga\AppData\Local\Continuum

In [2]:
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [3]:
train_df=downcast_dtypes(train_df)

In [4]:
train_df = train_df.T
train_df = train_df[6:] # because we need only sales 

In [5]:
sc = MinMaxScaler(feature_range = (0, 1))
train_df = sc.fit_transform(train_df)

In [6]:
timesteps=14
X_train = []
y_train = []
for i in range(timesteps, 1913):
    X_train.append(train_df[i-timesteps:i])
    y_train.append(train_df[i][0:30490]) 

X_train = np.array(X_train, dtype = 'float16')
y_train = np.array(y_train, dtype = 'float16')

X_train.shape,y_train.shape

((1899, 14, 30490), (1899, 30490))

In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(units =64,input_shape = (np.array(X_train).shape[1], np.array(X_train).shape[2])))
model.add(tf.keras.layers.Dense(30490))

model.compile(
  loss='mean_squared_error',
  optimizer=tf.keras.optimizers.Adam(0.001)
)
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 64)                7822080   
_________________________________________________________________
dense (Dense)                (None, 30490)             1981850   
Total params: 9,803,930
Trainable params: 9,803,930
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.fit(X_train, y_train, epochs = 10, batch_size = 10)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
1899/1899 [==============================] - 28s 15ms/sample - loss: 0.0141
Epoch 2/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0128
Epoch 3/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0125
Epoch 4/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0123
Epoch 5/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0122
Epoch 6/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0122
Epoch 7/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0122
Epoch 8/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0121
Epoch 9/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0121
Epoch 10/10
1899/1899 [==============================] - 23s 12ms/sample - loss: 0.0121


In [16]:
model.save('lstm_model')

## Test Data

In [17]:
model = tf.keras.models.load_model('lstm_model')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [9]:
inputs= train_df[-timesteps:] #14 days in tail
inputs = sc.transform(inputs)

In [10]:
X_test=[]
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)

In [11]:
predictions=model.predict(X_test)

In [12]:
predictions.shape

(1, 30490)

In [13]:
count = 0
forecast_day = 28
for i in range(forecast_day):
    inputs = train_df[-timesteps:]
    inputs = sc.transform(inputs)
    X_test = [inputs[0:timesteps]]
    X_test = np.array(X_test)
    predictions = model.predict(X_test)
    train_df = np.concatenate((train_df, predictions))
submission = train_df[forecast_day*(-1):]

In [27]:
new_sub = pd.DataFrame(submission.T)
new_sub.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,0.141931,0.128590,0.123224,0.118894,0.115873,0.113976,0.112777,0.112000,0.111480,0.111135,...,0.110272,0.110258,0.110248,0.110240,0.110234,0.110229,0.110225,0.110222,0.110220,0.110218
1,-0.017859,0.003833,0.014952,0.019399,0.022100,0.023892,0.025029,0.025769,0.026244,0.026559,...,0.027419,0.027440,0.027456,0.027469,0.027480,0.027489,0.027495,0.027501,0.027505,0.027509
2,0.118278,0.139145,0.143517,0.143968,0.143777,0.143390,0.142980,0.142631,0.142359,0.142145,...,0.141426,0.141409,0.141396,0.141385,0.141378,0.141372,0.141367,0.141363,0.141361,0.141359
3,0.165025,0.188335,0.192899,0.193004,0.193066,0.193083,0.193078,0.193054,0.192986,0.192936,...,0.192077,0.192043,0.192016,0.191995,0.191977,0.191963,0.191951,0.191942,0.191935,0.191931
4,0.041478,0.047647,0.055340,0.057557,0.059240,0.060934,0.062293,0.063316,0.064106,0.064704,...,0.066667,0.066727,0.066775,0.066813,0.066843,0.066867,0.066886,0.066900,0.066912,0.066920


In [28]:
new_sub['id']=pd.read_csv('../data/sales_train_validation.csv').id
# new_sub.columns=['id'] + ['F' + str(i + 1) for i in range(28)]


,0,1,2,3,4,5,6,7,8,9,...,19,20,21,22,23,24,25,26,27,id
0,0.141931,0.128590,0.123224,0.118894,0.115873,0.113976,0.112777,0.112000,0.111480,0.111135,...,0.110258,0.110248,0.110240,0.110234,0.110229,0.110225,0.110222,0.110220,0.110218,HOBBIES_1_001_CA_1_validation
1,-0.017859,0.003833,0.014952,0.019399,0.022100,0.023892,0.025029,0.025769,0.026244,0.026559,...,0.027440,0.027456,0.027469,0.027480,0.027489,0.027495,0.027501,0.027505,0.027509,HOBBIES_1_002_CA_1_validation
2,0.118278,0.139145,0.143517,0.143968,0.143777,0.143390,0.142980,0.142631,0.142359,0.142145,...,0.141409,0.141396,0.141385,0.141378,0.141372,0.141367,0.141363,0.141361,0.141359,HOBBIES_1_003_CA_1_validation
3,0.165025,0.188335,0.192899,0.193004,0.193066,0.193083,0.193078,0.193054,0.192986,0.192936,...,0.192043,0.192016,0.191995,0.191977,0.191963,0.191951,0.191942,0.191935,0.191931,HOBBIES_1_004_CA_1_validation
4,0.041478,0.047647,0.055340,0.057557,0.059240,0.060934,0.062293,0.063316,0.064106,0.064704,...,0.066727,0.066775,0.066813,0.066843,0.066867,0.066886,0.066900,0.066912,0.066920,HOBBIES_1_005_CA_1_validation


In [31]:
new_sub.columns=['F' + str(i + 1) for i in range(28)] + ['id']
new_sub.head()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F20,F21,F22,F23,F24,F25,F26,F27,F28,id
0,0.141931,0.128590,0.123224,0.118894,0.115873,0.113976,0.112777,0.112000,0.111480,0.111135,...,0.110258,0.110248,0.110240,0.110234,0.110229,0.110225,0.110222,0.110220,0.110218,HOBBIES_1_001_CA_1_validation
1,-0.017859,0.003833,0.014952,0.019399,0.022100,0.023892,0.025029,0.025769,0.026244,0.026559,...,0.027440,0.027456,0.027469,0.027480,0.027489,0.027495,0.027501,0.027505,0.027509,HOBBIES_1_002_CA_1_validation
2,0.118278,0.139145,0.143517,0.143968,0.143777,0.143390,0.142980,0.142631,0.142359,0.142145,...,0.141409,0.141396,0.141385,0.141378,0.141372,0.141367,0.141363,0.141361,0.141359,HOBBIES_1_003_CA_1_validation
3,0.165025,0.188335,0.192899,0.193004,0.193066,0.193083,0.193078,0.193054,0.192986,0.192936,...,0.192043,0.192016,0.191995,0.191977,0.191963,0.191951,0.191942,0.191935,0.191931,HOBBIES_1_004_CA_1_validation
4,0.041478,0.047647,0.055340,0.057557,0.059240,0.060934,0.062293,0.063316,0.064106,0.064704,...,0.066727,0.066775,0.066813,0.066843,0.066867,0.066886,0.066900,0.066912,0.066920,HOBBIES_1_005_CA_1_validation


In [33]:
new_sub = new_sub.reindex(columns=['id']+['F' + str(i + 1) for i in range(28)])
new_sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.141931,0.128590,0.123224,0.118894,0.115873,0.113976,0.112777,0.112000,0.111480,...,0.110272,0.110258,0.110248,0.110240,0.110234,0.110229,0.110225,0.110222,0.110220,0.110218
1,HOBBIES_1_002_CA_1_validation,-0.017859,0.003833,0.014952,0.019399,0.022100,0.023892,0.025029,0.025769,0.026244,...,0.027419,0.027440,0.027456,0.027469,0.027480,0.027489,0.027495,0.027501,0.027505,0.027509
2,HOBBIES_1_003_CA_1_validation,0.118278,0.139145,0.143517,0.143968,0.143777,0.143390,0.142980,0.142631,0.142359,...,0.141426,0.141409,0.141396,0.141385,0.141378,0.141372,0.141367,0.141363,0.141361,0.141359
3,HOBBIES_1_004_CA_1_validation,0.165025,0.188335,0.192899,0.193004,0.193066,0.193083,0.193078,0.193054,0.192986,...,0.192077,0.192043,0.192016,0.191995,0.191977,0.191963,0.191951,0.191942,0.191935,0.191931
4,HOBBIES_1_005_CA_1_validation,0.041478,0.047647,0.055340,0.057557,0.059240,0.060934,0.062293,0.063316,0.064106,...,0.066667,0.066727,0.066775,0.066813,0.066843,0.066867,0.066886,0.066900,0.066912,0.066920


## Output to submission.csv

In [34]:
sales = pd.read_csv('../data/sales_train_evaluation.csv')
sales.name = 'sales'

In [35]:
#Set actual equal to false if you want to top in the public leaderboard :P
actual = False
if actual == False:
    #Get the validation results(We already have them as less than one month left for competition to end)
    validation = sales[['id']+['d_' + str(i) for i in range(1914,1942)]]
    validation['id']=pd.read_csv('../data/sales_train_validation.csv').id
    validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
else:
    #Get the actual validation results
    valid['sold'] = valid_preds
    validation = valid[['id','d','sold']]
    validation = pd.pivot(validation, index='id', columns='d', values='sold').reset_index()
    validation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
    validation.id = validation.id.map(d_id).str.replace('evaluation','validation')

# #Get the evaluation results
# test['sold'] = eval_preds
# evaluation = test[['id','d','sold']]
# evaluation = pd.pivot(evaluation, index='id', columns='d', values='sold').reset_index()
# evaluation.columns=['id'] + ['F' + str(i + 1) for i in range(28)]
# #Remap the category id to their respective categories
# evaluation.id = evaluation.id.map(d_id)

#Prepare the submission
submit = pd.concat([validation,new_sub]).reset_index(drop=True)
submit.to_csv('submission_final_lstm.csv',index=False)

C:\Users\Yuga\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
